## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(10)
print(df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
54


### Вывод


1. Вероятно существуют пропуски в столбцах: days_employed, total_income

2. Видны аномалии в данных столбце days_employed на примере строки 4 (положительные значения на 2 порядка выше остальных отрицательных)

3. Значения в следующих столбцах могли бы иметь тип int64: days_employed, total_income
4. Значения в столбце education и purpose требуют приведения к общей форме

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#Проверяю на наличие пропусков и вывожу процент пропусков по столбцу   
print(df.isna().sum())
print(100 * df.isna().sum() / len(df))

#Проверяю на системность пропусков сравнением данных по невыплате в заполненных и незаполненных столбцах

nan_df = df[df.total_income.isnull()]

print(nan_df.groupby('income_type')['debt'].sum())
print(df.groupby('income_type')['debt'].sum())


#Вывожу медиану дохода по профессии, чтобы убедится, что можно уровнять

df_median = df.groupby('income_type').total_income.median()
print(df_median)

#Так как медианы по доходам оказались разными на порядок, заменяю пропуски медианами по каждому кредитору

median_unemployed = df['total_income'][df.income_type == 'безработный'].median()
df.loc[(df.income_type == 'безработный') & (pd.isna(df.total_income)),'total_income'] = median_unemployed
median_maternity = df['total_income'][df.income_type == 'в декрете'].median()
df.loc[(df.income_type == 'в декрете') & (pd.isna(df.total_income)),'total_income'] = median_maternity
median_goverment = df['total_income'][df.income_type == 'госслужащий'].median()
df.loc[(df.income_type == 'госслужащий') & (pd.isna(df.total_income)),'total_income'] = median_goverment
median_companion = df['total_income'][df.income_type == 'компаньон'].median()
df.loc[(df.income_type == 'компаньон') & (pd.isna(df.total_income)),'total_income'] = median_companion
median_pensioner = df['total_income'][df.income_type == 'пенсионер'].median()
df.loc[(df.income_type == 'пенсионер') & (pd.isna(df.total_income)),'total_income'] = median_pensioner
median_owner = df['total_income'][df.income_type == 'предприниматель'].median()
df.loc[(df.income_type == 'предприниматель') & (pd.isna(df.total_income)),'total_income'] = median_owner
median_emploee = df['total_income'][df.income_type == 'сотрудник'].median()
df.loc[(df.income_type == 'сотрудник') & (pd.isna(df.total_income)),'total_income'] = median_emploee
median_student = df['total_income'][df.income_type == 'студент'].median()
df.loc[(df.income_type == 'студент') & (pd.isna(df.total_income)),'total_income'] = median_student

# Так как количество отработанного времени в условиях задачи нигде не озвучено, 
# а в самих данных имеются аномалии, заменяю пропуски столбца на 0
df['days_employed'] = df['days_employed'].fillna(value = 0)

#Проверяю успещность замены
print(df.isnull().sum()) 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64
income_type
госслужащий         7
компаньон          30
пенсионер          35
предприниматель     0
сотрудник          98
Name: debt, dtype: int64
income_type
безработный           1
в декрете             1
госслужащий          86
компаньон           376
пенсионер           216
предприниматель       0
сотрудник

### Вывод

Обнаружены пропуски данных по доходу и количеству отработанных дней в одинаковом количестве, что составляет около 10 процентов от общего обьема данных. Так, как пропусков одинаковое количество по обоим столбцам, то вероятно это либо ошибка заполнении формы, либо ошибка переноса данных. В условиях задачи стоит определить зависимость выплаты кредита от других параметров, поэтому проверил отношение пропусков в данных к невыплате, отношение оказалось сопоставимо с данными по таблице без пропусков, поэтому решил что общей статистики это не повредит.
Перед тем как заполнить пропуски, проверил медианы зарплат по категории занятых, получил расхождения в порядок, поэтому заполнил пропущенные данные медианами по каждой категории.
Так как зависимость от количества отработанных дней мы не проверяем, а также учитывая аномалии в данных этой категории, пропуски в отработанных днях, заполнил нулевым значением.

### Замена типа данных

In [3]:
#Меняю тип в столбцах на целочисленный, потому что дни и доходы так проще считать
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Меняю тип в столбцах на целочисленный, потому что дни и доходы так проще считать и данные выглядят красивее

### Обработка дубликатов

In [4]:
print(df['purpose'].value_counts())
#print(df['family_status'].value_counts())
#print(df['education'].value_counts())
#print(df['income_type'].value_counts())


duplicates = df[df.duplicated(keep=False)]
duplicates.head()
df = df.drop_duplicates().reset_index(drop = True)
print(df.duplicated().sum())


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Необходима лемматизация по столбцам "purpose" и "education", но так как зависимость от уровня образования нам не нужна, с ней работать не будем

### Лемматизация

In [5]:

from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')
#Посмотрел что внутри
unique = df['purpose'].unique()
print(unique)

#Вызвал
from pymystem3 import Mystem
m = Mystem()
#Леммитизировал
def grouped_purpose(purpose):
    purposes = ['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        if lemma in purposes:
            return lemma
        
df['purpose'] = df['purpose'].apply(grouped_purpose)

df.head(10)


['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,жилье
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,жилье
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,жилье


### Вывод

Выделил 5 основных категорий, леммитизировал по ним 

### Категоризация данных

In [6]:
df.groupby('children').count().head
df['children'].replace([20], 2, inplace = True)
df['children'].replace([-1], 1, inplace = True)

def categorize_children(children):
    if children > 0:
        return 'C детьми'
    if children == 0:
        return 'Без детей'
    return 'Adult'

df['have_children'] = df.children.map(categorize_children)

def categorize_mariage(mariage):
    if mariage == 'женат / замужем':
        return 'В браке'
    return 'Не в браке'

df['mariage'] = df.family_status.map(categorize_mariage)

def categorize_income(income):
    if income <= 50000:
        return 'Бедный'
    if income <= 100000:
        return 'Низкий'
    if income <= 200000:
        return 'Средний'
    if income <= 300000:
        return 'Высокий'
    return 'Богатый'
   
df['income'] = df.total_income.apply(categorize_income) 

### Вывод

В соответствии с условиями добавил три с толбца с разбивкой на категории исходя из дохода, наличия детей и семейного положения

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
df.groupby('have_children')['debt'].sum() / df.groupby('have_children')['debt'].count()


have_children
C детьми     0.092070
Без детей    0.075353
Name: debt, dtype: float64

### Вывод

Клиенты с детьми допускают просрочку чаще

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
 df.groupby('mariage')['debt'].sum() / df.groupby('mariage')['debt'].count()

mariage
В браке       0.075421
Не в браке    0.088748
Name: debt, dtype: float64

### Вывод

Несостоящие в браке допускают просрочку чаще

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
df.groupby('income')['debt'].sum() / df.groupby('income')['debt'].count()

income
Бедный     0.061828
Богатый    0.071477
Высокий    0.070312
Низкий     0.080909
Средний    0.086174
Name: debt, dtype: float64

### Вывод

Клиенты со средним и низким доходом, допускают просрочку чаще

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
 df.groupby('purpose')['debt'].sum() / df.groupby('purpose')['debt'].count()

purpose
автомобиль      0.093547
жилье           0.069043
недвижимость    0.074610
образование     0.092177
свадьба         0.079657
Name: debt, dtype: float64

### Вывод

Клиенты с целями получения кредита на автомобиль и образование, допускают просрочку чаще

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Всё верно и точно сделано, как выводы, так и код написаны правильно


### Шаг 4. Общий вывод

Самый неудобный клиент - Не в браке, с детьми, с низким или средним доходом, желающий купить автомобиль или взять кредит на образование
Самый желанный клиент - В браке, без детей, с низким доходом, желающий приобрести жилье

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Хорошо, но можно было бы написать и больше выводов


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)
